# Download and Format Lawmatics Matters

This Notebook downloads all matters from Lawmatics, formats them into a tabular format, applies data types, then loads them into a BigQuery datastore table for further processing.

## Install Requirements

In [1]:
!pip install google.cloud google-cloud-secret-manager

## Retrieve Lawmatics API Token

In [2]:
#!/usr/bin/env python3
from google.cloud import secretmanager
from hashlib import sha1



def access_secret(project_id, secret_id, version_id="latest"):
    """
    Access a secret from Google Cloud Secret Manager

    Args:
        project_id: Your Google Cloud project ID
        secret_id: The ID of the secret
        version_id: The version of the secret (defaults to "latest")

    Returns:
        The secret value as a string
    """
    # Create the Secret Manager client
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version
    response = client.access_secret_version(request={"name": name})

    # Return the decoded secret
    return response.payload.data.decode("UTF-8")

# Example usage
if __name__ == "__main__":
    project_id = '566451752897'
    secret_id = 'mlo-lawmaticsApiKey'

    try:
        secret_value = access_secret(project_id, secret_id)
        secret_value_sha1 = sha1(secret_value.encode())
        secret_value_digest = secret_value_sha1.hexdigest()
        print(f"Secret retrieved successfully: (sha1) {secret_value_digest}")
    except Exception as e:
        print(f"Error accessing secret: {e}")

Secret retrieved successfully: (sha1) c32003bafa4577ecdbce1036cc7cedc098ae378a


## Get API JSON Data
Retrieve paginated data from a REST API using a non-expiring OAuth2 access token and transform it into a tabular format for analysis.

### Set up api request

#### Subtask:
Define the API endpoint, headers (including the access token), and any initial parameters for the first page of data.


In [3]:
api_endpoint = 'https://api.lawmatics.com/v1/prospects'
headers = {
    'Authorization': f'Bearer {secret_value}'
}
params = {
    'fields': 'all',
    'page': 1,
    'per_page': 100 # Or any other appropriate value based on API documentation
}

### Implement pagination logic

#### Subtask:
Handle the API's pagination, making requests for subsequent pages until all data is retrieved.


In [4]:
import requests
import time

all_data = []
page_num = 1
max_retries = 3
retry_delay = 5 # seconds

# api_endpoint is already defined in a previous cell
# headers and params are already defined in a previous cell

# Remove the Cookie header as it's not necessary according to API documentation
if 'Cookie' in headers:
    del headers['Cookie']


while True:
    params['page'] = page_num
    retries = 0
    while retries < max_retries:
        try:
            # Allow redirects by default (requests handles --location)
            response = requests.get(api_endpoint, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            page_data = response.json()

            # Check the structure of page_data based on the likely API response.
            # Assuming the API returns data in a key like 'data' and pagination info in 'meta'
            # If the API response structure is different, this part needs to be adjusted.
            if 'data' in page_data and isinstance(page_data['data'], list):
                current_page_items = page_data['data']
                if not current_page_items:  # Check if the 'data' list is empty
                    print("Empty data list received, assuming end of data.")
                    break # No more data

                all_data.extend(current_page_items)
                page_num += 1
                time.sleep(1) # Add a small delay to avoid overwhelming the API
                break # Break out of retry loop on success
            else:
                print("API response does not contain expected 'data' key with a list.")
                # Depending on the actual API response, you might want to
                # inspect page_data here to understand the structure.
                print(f"Response content: {page_data}")
                break # Exit if response structure is unexpected


        except requests.exceptions.RequestException as e:
            retries += 1
            print(f"Error during API request (Attempt {retries}/{max_retries}): {e}")
            if retries < max_retries:
                time.sleep(retry_delay)
            else:
                print("Max retries reached. Exiting.")
                break # Exit retry loop after max retries

    if retries == max_retries or ('data' in page_data and not page_data['data']):
        break # Exit main loop if max retries reached or no data received in the last successful attempt

print(f"Retrieved {len(all_data)} total records.")

Empty data list received, assuming end of data.
Retrieved 1802 total records.


## Process and convert data to tabular format

Convert the retrieved data from the API response (likely JSON) into a tabular format, such as a pandas DataFrame.


In [5]:
import pandas as pd

df = pd.DataFrame(all_data)
display(df.head())
display(df.info())

,id,type,attributes,relationships
0,14905792,prospect,"{'first_name': 'Joyce', 'last_name': 'Stacey',...","{'source': {'data': None}, 'stage': {'data': {..."
1,14884091,prospect,"{'first_name': 'C', 'last_name': 'F', 'case_ti...","{'source': {'data': {'id': '37839', 'type': 's..."
2,14882040,prospect,"{'first_name': 'Dee', 'last_name': 'Branch', '...","{'source': {'data': {'id': '37839', 'type': 's..."
3,14869489,prospect,"{'first_name': 'Ronald', 'last_name': 'Ramey',...","{'source': {'data': None}, 'stage': {'data': {..."
4,14868914,prospect,"{'first_name': 'Robert', 'last_name': 'Donovan...","{'source': {'data': None}, 'stage': {'data': {..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1802 non-null   object
 1   type           1802 non-null   object
 2   attributes     1802 non-null   object
 3   relationships  1802 non-null   object
dtypes: object(4)
memory usage: 56.4+ KB


None

### Extract nested data from columns

Extract relevant information from the 'attributes' and 'relationships' columns and create new columns in the DataFrame.

In [6]:
import pandas as pd
import re

# Extract data from the 'attributes' column if it exists
attributes_df = pd.DataFrame() # Initialize as empty
if 'attributes' in df.columns:
    attributes_df = df['attributes'].apply(pd.Series)
    df = df.drop(columns=['attributes'])


# Modified function to extract all relevant data from the 'relationships' column
# This function will now return a dictionary for each row
def extract_relationship_data_generic(relationship):
    extracted = {}
    if isinstance(relationship, dict):
        for key, value in relationship.items():
            if isinstance(value, dict) and 'data' in value:
                data_item = value['data']
                if isinstance(data_item, dict):
                    # For single related items, extract id and type
                    extracted[f"{key}_id"] = data_item.get('id')
                    extracted[f"{key}_type"] = data_item.get('type')
                elif isinstance(data_item, list):
                    # For lists of related items, join IDs and types
                    ids = [item.get('id') for item in data_item if isinstance(item, dict)]
                    types = [item.get('type') for item in data_item if isinstance(item, dict)]
                    # Use filter(None, ...) to remove None values before joining
                    extracted[f"{key}_ids"] = ','.join(filter(None, ids)) if ids else None
                    extracted[f"{key}_types"] = ','.join(filter(None, types)) if types else None
    return extracted


relationships_df_expanded = pd.DataFrame() # Initialize as empty
if 'relationships' in df.columns:
    # Apply the generic extraction function
    relationships_extracted_series = df['relationships'].apply(extract_relationship_data_generic)

    # Convert the Series of dictionaries into a DataFrame
    # This will create columns dynamically based on the keys present in the dictionaries
    relationships_df_expanded = pd.DataFrame(relationships_extracted_series.tolist())

    # Drop the original 'relationships' column
    df = df.drop(columns=['relationships'])


# Process 'custom_fields' column: Rename to 'custom_field_values' and keep as structured data
if 'custom_fields' in df.columns:
    df = df.rename(columns={'custom_fields': 'custom_field_values'})


# Final concatenation of all parts
# Ensure all parts have their index reset for correct concatenation.
all_parts = []

# Add the base dataframe (containing 'id', 'type', and other top-level columns after 'attributes' and 'relationships' drops)
# The 'custom_field_values' column (if present) is now part of this df
all_parts.append(df.reset_index(drop=True))

# Add the attributes dataframe if it's not empty
if not attributes_df.empty:
    all_parts.append(attributes_df.reset_index(drop=True))

# Add the relationships dataframe if it's not empty:
if not relationships_df_expanded.empty:
    all_parts.append(relationships_df_expanded.reset_index(drop=True))

# Note: custom_fields_df_expanded is no longer created or appended here,
# as the custom_field_values column is kept as structured data in the main df.

# Concatenate all parts along columns, filling missing values with NaN.
df = pd.concat(all_parts, axis=1, join='outer')

# Convert common date fields to datetime objects
date_columns_to_convert = [
    'created_at', 'updated_at', 'closed_at', 'last_activity_at',
    'intake_form_completed_at', 'first_contact_at', 'referral_date',
    'follow_up_date', 'opened_at', 'sent_at', 'due_at', 'start_date', 'end_date'
]

for col in date_columns_to_convert:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

# Convert NaN values to None for all columns
df = df.where(pd.notna(df), None)

# Function to clean column names for BigQuery compatibility
def clean_column_name(col_name):
    # Replace spaces with underscores
    col_name = col_name.replace(' ', '_')
    # Remove characters that are not alphanumeric or underscores
    col_name = re.sub(r'[^0-9A-Za-z_]', '', col_name)
    # Convert to lowercase
    col_name = col_name.lower()
    # Ensure it doesn't start with a number (BigQuery requirement)
    if col_name and col_name[0].isdigit():
        col_name = '_' + col_name
    # Truncate to BigQuery's max length (300 characters) if necessary
    col_name = col_name[:300]
    return col_name

# Apply column name cleaning to all columns in the DataFrame
df.columns = [clean_column_name(col) for col in df.columns]

# Display the first few rows of the updated DataFrame to verify the changes
display(df.head())
display(df.info())

,id,type,first_name,last_name,case_title,case_number,case_blurb,status,type_of_billing,name,...,source_id,source_type,campaign_id,campaign_type,originating_attorney_id,originating_attorney_type,owned_by_id,owned_by_type,company_id,company_type
0,14905792,prospect,Joyce,Stacey,"Stacey, Karl probate",300,None,hired,None,Joyce Stacey,...,None,None,None,None,None,None,None,None,None,None
1,14884091,prospect,C,F,None,None,None,pnc,None,C F,...,37839,source,92568,campaign,None,None,None,None,None,None
2,14882040,prospect,Dee,Branch,None,None,None,pnc,None,Dee Branch,...,37839,source,92568,campaign,None,None,None,None,None,None
3,14869489,prospect,Ronald,Ramey,Ramey Review,None,Review documents,pnc,None,Ronald Ramey,...,None,None,None,None,26070,user,23932,user,None,None
4,14868914,prospect,Robert,Donovan,Donovan Estate Plan,297,McCoy/Donovan Plan,hired,None,Robert C. Donovan,...,None,None,None,None,23932,user,23932,user,None,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Columns: 108 entries, id to company_type
dtypes: bool(1), datetime64[ns, UTC](2), float64(1), int64(4), object(100)
memory usage: 1.5+ MB


None

In [7]:
with pd.option_context('display.max_rows', None):
    print(df.dtypes)

id                                        object
type                                      object
first_name                                object
last_name                                 object
case_title                                object
case_number                               object
case_blurb                                object
status                                    object
type_of_billing                           object
name                                      object
email                                     object
phone                                     object
phone_number                              object
email_address                             object
address                                   object
birthdate                                 object
name_prefix                               object
middle_name                               object
name_suffix                               object
sub_status                                object
informal_name       

## Load the formatted data frame into a BQ table

In [8]:
import pandas_gbq

project_id = 'www-prod-389819'
table_id = 'mlo_kpi.staging_lm_matters'

# Load the DataFrame to BigQuery, replacing existing data
pandas_gbq.to_gbq(
    df,
    table_id,
    project_id=project_id,
    if_exists='replace'
)

print(f"DataFrame loaded to {table_id} in project {project_id} successfully, replacing existing data.")

/usr/local/lib/python3.12/dist-packages/pandas_gbq/schema/pandas_to_bigquery.py:157: UserWarning: Could not determine the type of columns: custom_fields
  warnings.warn(msg)
100%|██████████| 1/1 [00:00<00:00, 7294.44it/s]

DataFrame loaded to mlo_kpi.staging_lm_matters in project www-prod-389819 successfully, replacing existing data.
